# Create a convolutional neural network (CNN)
model = tf.keras.quential([
    Cotf.keras.layers.nv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    Matf.keras.layers.xPooling2D(2, 2),
    Cotf.keras.layers.nv2D(64, (3, 3), activation='relu'),
    Matf.keras.layers.xPooling2D(2, 2),
    Cotf.keras.layers.nv2D(128, (3, 3), activation='relu'),
    Matf.keras.layers.xPooling2D(2, 2),
    Fltf.keras.layers.atten(),
    Detf.keras.layers.nse(128, activation='relu'),
    Detf.keras.layers.nse(len(clothing_names), activation='softmax')
])tf

In [ ]:
#----------------------------IMPORTS----------------------------
import cv2
import numpy as np
import os
import random
import pickle
import tensorflow as tf
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt

In [ ]:
path = 'D:\Senior II\Graduation Project\Datasets\Clothes'
df = pd.read_csv(path + '/images.csv')
df.head()

In [ ]:
#Remove all entries with label = Not sure
df = df[df['label'] != 'Not sure']
#Remove colums: sender_id and kids
df = df.drop(['sender_id', 'kids'], axis=1)
df.head()

In [ ]:
#Extract the unique labels
class_names = df['label'].unique()
print(f'There are {class_names.shape[0]} unique classes')
class_ids = dict()
for i in range(len(class_names)):
    class_ids[class_names[i]] = i

In [ ]:
image_width = image_height = 128

In [ ]:
#Upload photos to a numpy array
images_train = []
images_labels_train = []
images_test = []
images_labels_test = []
# for i in range(len(df['image'])):
for i in range(1000):
    #Check if df['image'][i] is in
    if (i in df['image'].keys()):
        img = cv2.imread(path + '/' + df['image'][i] + '.jpg')
        #Check if image is not None
        if (img is not None):
            images_train.append(cv2.resize(img, (image_width, image_height)))
            images_labels_train.append(class_ids[df['label'][i]])

for j in range(300):
    #Check if df['image'][i] is in
    if ((i + j) in df['image'].keys()):
        img = cv2.imread(path + '/' + df['image'][i + j] + '.jpg')
        #Check if image is not None
        if (img is not None):
            images_test.append(cv2.resize(img, (image_width, image_height)))
            images_labels_test.append(class_ids[df['label'][i + j]])

            
x_train = np.array(images_train)
y_train = np.array(images_labels_train)

x_test = np.array(images_test)
y_test = np.array(images_labels_test)

In [ ]:
#Show an image from x_train
plt.imshow(x_train[401])
plt.show()

In [ ]:
#Number of training examples
m = x_train.shape[0]
m_test = x_test.shape[0]

#Reshape the training and test examples
# x_train = x_train.reshape(m, -1)
# x_test = x_test.reshape(m_test, -1)

print(f'x_train.shape = {x_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'x_test.shape = {x_test.shape}')
print(f'y_test.shape = {y_test.shape}')

In [117]:
# Create a convolutional neural network (CNN)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (image_width, image_height,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='linear')
])

# Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
)

In [ ]:
# Train the model on the training data
# history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))
history = model.fit(x_train, y_train, epochs=10, batch_size=32)

In [ ]:
#Plot loss vs epochs
plt.plot(history.history['loss'])
plt.show()

In [ ]:
_, train_accuracy = model.evaluate(x_train, y_train)
_, test_accuracy = model.evaluate(x_test, y_test)
print("Train Accuracy = %.2f" % (train_accuracy*100),'% with', m, 'training examples')
print("Test Accuracy = %.2f" % (test_accuracy*100),'% with', m_test, 'test examples')

In [ ]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)
train_count_correct = 0
test_count_correct = 0
for i in range(m):
    predicted = np.argmax(train_predictions[i])
    if (predicted == y_train[i]):
        train_count_correct += 1
train_accuracy = train_count_correct / m

for i in range(m_test):
    predicted = np.argmax(test_predictions[i])
    if (predicted == y_test[i]):
        test_count_correct += 1
test_accuracy = test_count_correct / m_test

print("Train Accuracy = %.2f" % (train_accuracy*100),'% with', m, 'training examples')
print("Test Accuracy = %.2f" % (test_accuracy*100),'% with', m_test, 'test examples')


In [ ]:
prediction = model.predict(x_train[np.newaxis,0])
prediction
class_names[np.argmax(prediction)]

In [118]:
del model

In [ ]:
# Save the trained model
pickle.dump(model, open('clothing_detector.pkl', 'wb'))

In [ ]:
# Plot a summary of your model's architecture
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)